In [ ]:
from ultralytics import YOLO

# Carga el modelo ya entrenado (puedes usar 'best.pt' o 'last.pt')
model = YOLO('/home/luis/Documents/UTEC/Computer_Vision/models/runs/pose/train322/weights/best.pt')

# Ejecuta la predicción sobre una imagen
results = model.predict(source='../preprocessing/data/basketball-court-keypoints1-1/test/images/faasdasd2_275_jpg.rf.7abe060a830772a86c63bfa41ce2a9a6.jpg', task='pose', imgsz=640, save=True)

for res in results:  # puede contener varios objetos, aunque en tu caso 1 corte = 1 objeto
    kp = res.keypoints  # objeto Keypoints
    coords = kp.xy         # coordenadas en píxeles (N_points × 2)
    coords_norm = kp.xyn   # coordenadas normalizadas (0–1)
    confs = kp.conf        # nivel de confianza por keypoint
    print(coords_norm)
    # Opcional: res.show() para visualizar en pantalla
    res.show()


image 1/1 /home/luis/Documents/UTEC/Computer_Vision/proyecto/data/basketball-court-keypoints1-1/test/images/faasdasd2_275_jpg.rf.7abe060a830772a86c63bfa41ce2a9a6.jpg: 384x640 1 court, 37.1ms
Speed: 4.3ms preprocess, 37.1ms inference, 97.3ms postprocess per image at shape (1, 3, 384, 640)
Results saved to /home/luis/Documents/UTEC/Computer_Vision/runs/pose/predict2
tensor([[[0.0000, 0.0000],
         [0.0000, 0.0000],
         [0.0000, 0.0000],
         [0.0000, 0.0000],
         [0.0000, 0.0000],
         [0.0000, 0.0000],
         [0.5765, 0.3628],
         [0.5972, 0.4148],
         [0.2458, 0.5059],
         [0.0000, 0.0000],
         [0.0000, 0.0000],
         [0.0000, 0.0000],
         [0.0000, 0.0000],
         [0.5805, 0.7194],
         [0.4355, 0.4476],
         [0.0000, 0.0000],
         [0.0000, 0.0000],
         [0.5302, 0.5565],
         [0.4131, 0.7033],
         [0.0000, 0.0000],
         [0.0000, 0.0000],
         [0.0000, 0.0000],
         [0.0000, 0.0000],
         [0

/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/luis/snap/code/common/.cache/gio-modules/libgiolibproxy.so
eog: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_init, version GLIBC_PRIVATE


In [ ]:
from ultralytics import YOLO
import cv2

# 🚨 Carga ambos modelos ya entrenados
court_model = YOLO("/home/luis/Documents/UTEC/Computer_Vision/proyecto/models/runs/pose/train3/weights/best.pt")
people_model = YOLO("/home/luis/Documents/UTEC/Computer_Vision/proyecto/models/runs/detect/train/weights/best.pt")

# Ejecuta inferencia en la imagen
img_path = "../preprocessing/data/detect/basketball-ball-3/val/images/davis4_jpg.rf.968976f803d763a4765c2858ea2ffa97.jpg"
base_img = cv2.imread(img_path)
res_court = court_model.predict(source=img_path, task='pose', imgsz=640, save=False)[0]
print(res_court)  # Imprime las coordenadas de los keypoints
annotated = res_court.plot(img = base_img.copy())

res_people = people_model.predict(source=img_path, task='detect', imgsz=640, save=False)[0]

annotated = res_people.plot(img = annotated)

cv2.imshow("Court + People", annotated)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite("combinado.jpg", annotated)  # Guardado opcional


image 1/1 /home/luis/Documents/UTEC/Computer_Vision/proyecto/train/../preprocessing/data/detect/basketball-ball-3/val/images/davis4_jpg.rf.968976f803d763a4765c2858ea2ffa97.jpg: 384x640 1 p1, 9.4ms
Speed: 0.8ms preprocess, 9.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: ultralytics.engine.results.Keypoints object
masks: None
names: {0: 'p1'}
obb: None
orig_img: array([[[  3,   3,   3],
        [  3,   3,   3],
        [  3,   3,   3],
        ...,
        [  5,   4,   6],
        [  5,   4,   6],
        [  5,   4,   6]],

       [[  3,   3,   3],
        [  3,   3,   3],
        [  3,   3,   3],
        ...,
        [  5,   4,   6],
        [  5,   4,   6],
        [  5,   4,   6]],

       [[  3,   3,   3],
        [  3,   3,   3],
        [  3,   3,   3],
        ...,
        [  5,   4,   6],
        [  5,   4,   6],
        [  5,   4,   6]],

